In [119]:
import os, sys
from google.colab import drive
drive.mount('/content/drive')
my_path = '/content/notebooks'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [120]:
!pip install konlpy


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [121]:
import re
import pandas as pd
import numpy as np 
# import matplotlib.pyplot as plt
# import urllib.request
from collections import Counter
from konlpy.tag import Mecab
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
# test
review_data_frm_04 = pd.read_excel('/content/drive/MyDrive/Crawling/제주특별자치도/제주시/식당정보/제주특별자치도_제주시_연동_리뷰전체모음_04.xlsx')

In [200]:
class psng_predict_once() :
  def __init__(self , df) :
    self.df = df
    
    try :
      self.review = df.리뷰
    except :
      try :
        self.review = df.REVIEW
      except :
        print('DataFrame에 리뷰 or Review 칼럼이 존재하지 않거나, DataFrame 형식이 아닙니다.')
        return
    
    # 경로 확인
      # 모델 로드
    self.loaded_model = load_model(r'/content/drive/MyDrive/data_review/archive/new_model_01_v2.h5')
      #  텍스트 데이터를 토큰화하고 단어 사전을 구축하기위한 학습시 사용한 train_data 로드
    train_data = pd.read_csv(r'/content/drive/MyDrive/data_review/archive/train_data_main.csv' , encoding = 'euc-kr')
      # 모델 성능 확인용 test_data
    test_data = pd.read_csv(r'/content/drive/MyDrive/data_review/archive/test_data_main.csv' , encoding = 'euc-kr')

    # 데이터 정규화
      # array("['a','b','c']") -> array(list['a','b','c'])
    X_train = train_data['tokenized'].apply(eval).values
    self.X_test = test_data['tokenized'].apply(eval).values
    self.y_test = np.array(test_data['label'].values)
    
    self.X_test_tkd = None

    # tokenized option
    vocab_size = 87912
      # OOV 
    self.tokenizer = Tokenizer(vocab_size, oov_token = 'OOV') 
    self.tokenizer.fit_on_texts(X_train)

    self.max_len = 250

  def preprocessing(self , R_frm) :
    
    stopwords = [ '도', '는', '다', '의', '가', '이', '은', 
                  '한', '에', '하', '고', '을', '를', '인', '듯', 
                  '과', '와', '네', '들', '듯', '지', '임', '게', 
                  '는', '이', '했', '슴', '음', '것', '거', '로',
                  '들', '거', '곳', '분', '원', '더', '왜', '해',
                  '수', '할', '그', '함', '돈', '번', '두' ,'개',
                  '건', '내', '저', '만', '갈', '걸', '제', '명', 
                  '분', 
                  '인데', '이가', '했었','해서',
                  '습니다', '했는데', '입니다' ]

    # 정규 표현식 수행
    R_frm['리뷰'] = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','', R_frm['리뷰'])
    
    # 공백은 Null 값으로 변경
    if R_frm['리뷰'] == '' :
      R_frm['리뷰'] = None
      self.R_frm_tokenized_value = None
      return self.R_frm_tokenized_value
    else :
      # konlpy okt -> 문장 토큰화 (모델 개선시 : konlpy 동사 형용사 명사 사전 업데이트)
      encoded = okt.pos(R_frm['리뷰'])
      # print(encoded)
      encoded = [word for word,shape in encoded if shape in ['Verb' , 'Adjective' , 'Noun' , 'VerbPrefix'] if word not in stopwords]
      self.R_frm_tokenized_value = encoded
      
      return self.R_frm_tokenized_value

  def model_test(self) :
    tokenizer = self.tokenizer
    
    X_test = tokenizer.texts_to_sequences(self.X_test)
    y_test = self.y_test
    X_test = pad_sequences(X_test, maxlen=self.max_len)

    print("\n 테스트 정확도: %.4f" % (self.loaded_model.evaluate(X_test, y_test)[1]))

  def word_index(self) :
    tokenizer = self.tokenizer
    
    print(tokenizer.word_index)

  def predict(self) :

    # 리뷰 하나씩 예측

    R_frm = self.df.copy()
    reviews = self.review.tolist()
    max_len = self.max_len
    tokenizer = self.tokenizer
    result = []

    for num , review in enumerate(reviews) :
      
      R_pred = self.preprocessing(R_frm.copy().iloc[num])
      
      if R_pred == None :
        result.append('-1')
      else :
        pred = tokenizer.texts_to_sequences([R_pred])
        pred = pad_sequences(pred , maxlen=max_len)
        # print(pred)
        score = float(self.loaded_model.predict(pred))

        print(num)
        print(review)
        if score > 0.9 :
          result.append('0')
        elif (score < 0.1) :
          result.append('1')
        else :
          result.append('2')
        print()

    return result



In [201]:
t = psng_predict_once(review_data_frm_04[:5])

In [ ]:
t.predict()

## test

In [124]:
okt = Okt()

In [111]:
loaded_model = load_model(r'/content/drive/MyDrive/data_review/archive/new_model_01_v2.h5')

In [8]:
train_data = pd.read_csv(r'/content/drive/MyDrive/data_review/archive/train_data_main.csv' , encoding = 'euc-kr')

In [9]:
test_data = pd.read_csv(r'/content/drive/MyDrive/data_review/archive/test_data_main.csv' , encoding = 'euc-kr')

In [97]:
# array("['a','b','c']") -> array(list['a','b','c'])
X_train = train_data['tokenized'].apply(eval).values

In [98]:
X_train

array([list(['난', '별로', '달다', '먼저', '왔는데도', '늦게', '온', '테이블', '세팅', '나가고', '블로그', '맛', '집', '떠서', '가봤는데', '개인', '차', '큰', '같음']),
       list(['짜가', '정말', '정말', '맛있었어요', '강', '추합니다', '또', '가고', '싶을', '정도']),
       list(['어어', '어어어어', '무', '기대해서', '런가', '별로', '였다', '딱히', '특별하지', '않은', '맛', '결정', '패티', '그랬다']),
       ..., list(['빵', '겁나', '맛없음', '커피']),
       list(['음료', '미지', '밍밍', '증편', '떡', '구이', '따뜻하긴', '내어주시는', '발사믹', '강하고', '떡', '안', '어울려요', '오픈', '기간', '방문', '지금', '좀', '나아졋길', '바랍니다']),
       list(['고등학생', '때', '진짜', '맛있게', '먹었던', '같은데', '사', '년', '지난', '후', '지금', '먹어', '보니', '달다', '악덕', '사서', '옆', '편의점', '테이블', '맥주', '사서', '먹으면', '꿀이긴'])],
      dtype=object)

In [100]:
X_test = test_data['tokenized'].apply(eval).values

In [57]:
y_test = np.array(test_data['label'].values)

In [50]:
X_test

array([list(['맛', '괜찮은지', '모르는데', '계산', '하려면', '짜증', '캐셔', '좀', '늘려서', '해소', '해주면', '좋을', '성질', '급한', '사람', '그냥', '안', '사', '먹고', '말']),
       list(['주위', '유명한', '집', '닫아서', '갔는데', '영업', '하시냐고', '주방', '할머니', '여쭤', '보니', '그냥', '들어오면', '되지', '물어봐라고', '미', '간', '찌푸리고', '짜증', '내며', '소리', '지르네요', '분노조절', '장애인', '맛', '짬뽕', '외', '성비', '최악', '결론', '불친절', '성비', '최악']),
       list(['갑작스럽게', '급', '방문', '기대', '일도', '안', '하고', '가서', '런가', '맛있게', '잘', '먹었다', '여긴', '스텔라', '피자', '꼭', '먹어야', '된다']),
       ...,
       list(['바다', '가까운데', '뷰', '좋은', '아니고', '안', '좋은', '아니고', '애매함', '자리', '쟁탈전', '치열하고', '음료', '양', '퀄리티', '비해', '매우', '비싼', '편']),
       list(['베트남', '맥주', '거리', '같은', '떠들썩하고', '와글와글', '젊은', '분위기']),
       list(['사악한', '가격', '대체재', '없으므로', '찾게', '되는', '가게', '맛', '변하지', '않았으면'])],
      dtype=object)

In [58]:
y_test

array([0, 0, 1, ..., 0, 1, 1])

In [170]:
review_data_frm_04.리뷰.info()

<class 'pandas.core.series.Series'>
RangeIndex: 17509 entries, 0 to 17508
Series name: 리뷰
Non-Null Count  Dtype 
--------------  ----- 
17509 non-null  object
dtypes: object(1)
memory usage: 136.9+ KB


In [177]:
new_sentence = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','', R_frm.iloc[0].리뷰)
new_sentence

'좋아요'

In [67]:
review_data_frm_04 = pd.read_excel('/content/drive/MyDrive/Crawling/제주특별자치도/제주시/식당정보/제주특별자치도_제주시_연동_리뷰전체모음_04.xlsx')

In [86]:
R_frm = review_data_frm_04.copy()

In [125]:
R_frm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17509 entries, 0 to 17508
Data columns (total 12 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   리뷰         17509 non-null  object
 1   리뷰이미지url   6386 non-null   object
 2   작성자프로필     17509 non-null  object
 3   My플레이스     17509 non-null  object
 4   작성자이름      17509 non-null  object
 5   리뷰작성수      17509 non-null  object
 6   리뷰작성일      17509 non-null  object
 7   식당방문수      17509 non-null  object
 8   리뷰인증방식     17509 non-null  object
 9   식당id       17509 non-null  int64 
 10  사업장명       17509 non-null  object
 11  tokenized  17509 non-null  object
dtypes: int64(1), object(11)
memory usage: 1.6+ MB


In [93]:
R_frm['리뷰'] = R_frm.리뷰.str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","", regex=True) # 정규 표현식 수행
R_frm['리뷰'].replace('', np.nan, inplace=True) # 공백은 Null 값으로 변경
R_frm['리뷰'] = R_frm['리뷰'].astype(str)
stopwords = ['도', '는', '다', '의', '가', '이', '은', 
            '한', '에', '하', '고', '을', '를', '인', '듯', 
            '과', '와', '네', '들', '듯', '지', '임', '게' , 
            '는' ,'이','인데','이가','했었','했','습니다','슴','음' ,
            '것' ,'거' ,'로' ,'들' ,'거' ,'것' ,'곳' , '분' , '원' ,
            '입니다' ,'해서' ,'더' ,'왜' ,'수' ,'할' , '그' ,'함' ,'돈' ,'번' ,
            '두' ,'개' ,'했는데' ,'건' ,'내' , '저' ,'만' ,'갈','걸','제','명' ,
            '분' ,'해']
R_frm['tokenized'] = R_frm['리뷰'].apply(okt.pos)
R_frm['tokenized'] = R_frm['tokenized'].apply(lambda x: [word for word,shape in x if shape in ['Verb' , 'Adjective' , 'Noun' , 'VerbPrefix'] if word not in stopwords])

In [94]:
R_pred = R_frm['tokenized'].values

In [132]:
# 한 문장씩 값을 반환해야하기에 배열을 쪼개주자
t = R_frm['tokenized'].values

In [135]:
for i in t[:3] :
  print(i)

['좋아요']
['피낫', '버터', '쿠키앤크림', '맛있어요']
['좋아요']


In [95]:
R_pred

array([list(['좋아요']), list(['피낫', '버터', '쿠키앤크림', '맛있어요']), list(['좋아요']),
       ..., list(['굿']), list(['대화', '하기', '좋아요']), list(['굿'])],
      dtype=object)

In [101]:
# vocab_size = 46827
vocab_size = 87912
# OOV
tokenizer = Tokenizer(vocab_size, oov_token = 'OOV') 
# X_train - 훈련 때 사용했던 data 필요
tokenizer.fit_on_texts(X_train)
# X_train - 예측시, 예측 데이터 필요
# X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)
R_pred = tokenizer.texts_to_sequences(R_pred)

In [ ]:
tokenizer.word_index

In [ ]:
R_pred[:3]

In [ ]:
X_test[:3]

In [104]:
max_len = 250

In [105]:
X_test = pad_sequences(X_test, maxlen=max_len)
R_pred = pad_sequences(R_pred , maxlen=max_len)

In [109]:
X_test.shape

(34966, 250)

In [108]:
R_pred.shape

(17509, 250)

In [234]:
print(R_pred)

[[    0     0     0 ...     0     0    57]
 [    0     0     0 ...   605 36817    29]
 [    0     0     0 ...     0     0    57]
 ...
 [    0     0     0 ...     0     0   801]
 [    0     0     0 ...  1065   194    57]
 [    0     0     0 ...     0     0   801]]


In [66]:
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))

1093/1093 [==============================] - 86s 78ms/step - loss: 0.1658 - acc: 0.9370

 테스트 정확도: 0.9370


In [203]:
loaded_model.predict(R_pred)

548/548 [==============================] - 41s 75ms/step


array([[0.92867976],
       [0.7815909 ],
       [0.92867976],
       ...,
       [0.8745687 ],
       [0.6947712 ],
       [0.8745687 ]], dtype=float32)

In [ ]:
for num , review in enumerate(review_data_frm_04.리뷰.tolist()[:5]) :
  print(num)
  print(review)
  print(float(loaded_model.predict(R_pred[num])))
  print()

# 전체 돌리기

In [238]:
class psng_predict_all() :
  def __init__(self , df) :
    self.df = df
    
    try :
      self.review = df.리뷰
    except :
      try :
        self.review = df.REVIEW
      except :
        print('DataFrame에 리뷰 or Review 칼럼이 존재하지 않거나, DataFrame 형식이 아닙니다.')
        return
    
    # 경로 확인
      # 모델 로드
    self.loaded_model = load_model(r'/content/drive/MyDrive/data_review/archive/new_model_01_v2.h5')
      #  텍스트 데이터를 토큰화하고 단어 사전을 구축하기위한 학습시 사용한 train_data 로드
    train_data = pd.read_csv(r'/content/drive/MyDrive/data_review/archive/train_data_main.csv' , encoding = 'euc-kr')
      # 모델 성능 확인용 test_data
    test_data = pd.read_csv(r'/content/drive/MyDrive/data_review/archive/test_data_main.csv' , encoding = 'euc-kr')

    # 데이터 정규화
      # array("['a','b','c']") -> array(list['a','b','c'])
    X_train = train_data['tokenized'].apply(eval).values
    self.X_test = test_data['tokenized'].apply(eval).values
    self.y_test = np.array(test_data['label'].values)
    
    self.X_test_tkd = None

    # tokenized option
    vocab_size = 87912
      # OOV 
    self.tokenizer = Tokenizer(vocab_size, oov_token = 'OOV') 
    self.tokenizer.fit_on_texts(X_train)

    self.max_len = 250

  def preprocessing(self) :
    R_frm = self.df.copy()
    
    # 정규 표현식 수행
    R_frm['리뷰'] = R_frm.리뷰.str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","", regex=True) 
    # 공백은 Null 값으로 변경
    R_frm['리뷰'].replace('', np.nan, inplace=True) 
    R_frm['리뷰'] = R_frm['리뷰'].astype(str)
    stopwords = ['도', '는', '다', '의', '가', '이', '은', 
                 '한', '에', '하', '고', '을', '를', '인', '듯', 
                 '과', '와', '네', '들', '듯', '지', '임', '게', 
                 '는', '이', '했', '슴', '음', '것', '거', '로',
                 '들', '거', '곳', '분', '원', '더', '왜', '해',
                 '수', '할', '그', '함', '돈', '번', '두' ,'개',
                 '건', '내', '저', '만', '갈', '걸', '제', '명', 
                 '분', 
                 '인데', '이가', '했었','해서',
                 '습니다', '했는데', '입니다']

    # konlpy okt -> 문장 토큰화 (모델 개선시 : konlpy 동사 형용사 명사 사전 업데이트)
    R_frm['tokenized'] = R_frm['리뷰'].apply(okt.pos)
    R_frm['tokenized'] = R_frm['tokenized'].apply(lambda x: [word for word,shape in x if shape in ['Verb' , 'Adjective' , 'Noun' , 'VerbPrefix'] if word not in stopwords])
    R_pred = R_frm['tokenized'].values
    return R_pred

  def model_test(self) :
    tokenizer = self.tokenizer
    
    X_test = tokenizer.texts_to_sequences(self.X_test)
    y_test = self.y_test
    X_test = pad_sequences(X_test, maxlen=self.max_len)

    print("\n 테스트 정확도: %.4f" % (self.loaded_model.evaluate(X_test, y_test)[1]))

  def word_index(self) :
    tokenizer = self.tokenizer
    
    print(tokenizer.word_index)

  def predict(self) :
    R_pred = self.preprocessing()
    max_len = self.max_len
    tokenizer = self.tokenizer
    result = []
    pred = tokenizer.texts_to_sequences(R_pred)
    pred = pad_sequences(pred , maxlen=max_len)
    print(np.array(pred))
    score = self.loaded_model.predict(pred)
    return score
  
  def prediction(self) :
    result = []
    score = self.predict()
    
    for num in range(len(score)) :
      if score[num][0] > 0.9 :
        result.append('0')
      elif score[num][0] < 0.1 :
        result.append('1')
      else :
        result.append('2')

    return result


    
  


In [239]:
oo = psng_predict_all(review_data_frm_04[:5])

In [240]:
oo.prediction()

[[    0     0     0 ...     0     0    57]
 [    0     0     0 ...   605 36817    29]
 [    0     0     0 ...     0     0    57]
 [    0     0     0 ...     0     0   801]
 [    0     0     0 ...     0     0   801]]
1/1 [==============================] - 1s 812ms/step


['0', '2', '0', '2', '2']